GPU_number<br/>
batch_size = 64<br/>
CH_N = 3<br/>
def write_file()

   # Initial
   https://arxiv.org/pdf/1803.09820.pdf

In [2]:
import os

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # or any {'0', '1', '2'}
import tensorflow as tf

tf.get_logger().setLevel('ERROR')
from tensorflow.python.client import device_lib
import tensorflow.keras as keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import time
import random as rd
import cv2
import glob, shutil, openpyxl, datetime

from tensorflow import float32
import math
import visualkeras
from PIL import ImageFont

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
device = tf.config.list_physical_devices('GPU')[0]
tf.config.experimental.set_memory_growth(
    device, True
)

Version:  2.7.0
Eager mode:  True
GPU is available


I think this code below is used to create folders and xlx files

In [3]:
root = ''
date = str(datetime.date.today())
foldername = 'cabbage_' + date
dirp = root + foldername
if not os.path.exists(dirp):
    os.mkdir(dirp)
    ulogdir = dirp + '/logs'
    os.mkdir(dirp + '/Models')
    os.mkdir(dirp + '/Val')
    os.mkdir(dirp + '/Test')

# ## TODO ask Ida what is the template is ?
source = r'template.xlsx'
dst = root + r'cabbage.xlsx'
sheet_name = 'cabbage_3c_'

if not os.path.exists(dst):
    shutil.copyfile(source, dst)
workbook = openpyxl.load_workbook(dst)
sheet = workbook['sheet1']
workbook.copy_worksheet(sheet)
sh_name = workbook.sheetnames  # 获取所有sheet
sh = workbook[sh_name[-1]]
sh.title = sheet_name
sh['A1'] = sheet_name
sh['C1'] = foldername
sh_name = workbook.sheetnames
sheet_name = sh_name[-1]
workbook.save(dst)

print('dirp = \'' + dirp + '\'')
print('dst = \'' + dst + '\'')
print('sheet_name = \'' + sheet_name + '\'')
# print("tensorboard --logdir=\"D:/py_project/twcc/{}\" --bind_all".format(foldername))

dirp = 'cabbage_2022-05-09'
dst = 'cabbage.xlsx'
sheet_name = 'cabbage_3c_12'


In [4]:
gpu = tf.config.experimental.list_physical_devices(device_type='GPU')
assert len(gpu) == 1
tf.config.experimental.set_memory_growth(gpu[0], True)

# TODO this should be made as CONSTANT
BATCH_SIZE = 16 #64
CH_N = 3
imgs = 256
print(BATCH_SIZE)


16


   # Dataset

In [9]:
# CHANGE THE LOCATION OF TRAINING & VALIDATION IMAGES
imgfiles = [f for f in glob.glob(r'/home/kuro/Documents/cabbage/FOLD1_3CLUSTER/F1C1_IMG_3/*.png')]
mskfiles = [f for f in glob.glob(r'/home/kuro/Documents/cabbage/FOLD1_3CLUSTER/F1C1_MASK_3/*.png')]

imgfiles.sort()
mskfiles.sort()
img = cv2.imread(imgfiles[5800])
mask = cv2.imread(mskfiles[5901])
# print(np.unique(img))
print(np.unique(mask))
print(len(imgfiles), len(mskfiles))

[  0 255]
10800 10800


In [ ]:
img = cv2.imread(imgfiles[500])
print(max(np.unique(img)))
print(img.shape)
msk = cv2.imread(mskfiles[500])
print(max(np.unique(msk)))
print(msk.shape)

In [6]:
# TODO what is 2160?
n_pic = len(imgfiles) // 2160
whole_imgl = [{} for _ in range(n_pic)]
whole_mskl = [{} for _ in range(n_pic)]
for r, i in enumerate(zip(imgfiles, mskfiles)):
    whole_imgl[r % n_pic][int(i[0].split('/')[-1].split('_')[0])] = (i[0])
    whole_mskl[r % n_pic][int(i[0].split('/')[-1].split('_')[0])] = (i[1])
w_imgl = [[x.get(i) for i in range(1, len(x) + 1, 1)] for x in whole_imgl]
w_mskl = [[x.get(i) for i in range(1, len(x) + 1, 1)] for x in whole_mskl]
print(len(w_imgl))

5


In [19]:
def imggen(imgpath, mskpath, stop, imgs, aug=False, whole=False):
    i = 0
    if whole:
        i = 1
    while i < stop:
        #img = tiffile.imread(imgpath[i].decode())
        img = tf.io.read_file(imgpath[i])
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.divide(img, 255)
        img = tf.image.convert_image_dtype(img, tf.float32)
        # img = tf.image.resize(img,(imgs,imgs))
        msk = tf.io.read_file(mskpath[i])
        msk = tf.image.decode_jpeg(msk, channels=1)
        msk = tf.image.convert_image_dtype(msk, tf.float32)
        # msk = tf.image.resize(msk,(imgs,imgs))
        msk = tf.math.logical_and(msk < 255, msk > 0)
        msk = tf.cast(msk, tf.float32)

        if aug:
            if rd.random() > 0.5:
                img = tf.image.flip_left_right(img)
                msk = tf.image.flip_left_right(msk)
            if rd.random() > 0.5:
                img = tf.image.flip_up_down(img)
                msk = tf.image.flip_up_down(msk)
            if rd.random() > 0.6:
                img = tf.image.random_brightness(img, 0.3)

        img = tf.image.per_image_standardization(img)
        #if CH_N<4:
        #img = tf.slice(img,[0,0,0],[-1,-1,CH_N])

        i += 1
        yield img, msk

In [8]:
def fixx(img, msk):
    img.set_shape([imgs, imgs, CH_N])
    msk.set_shape([imgs, imgs, 1])
    return img, msk

In [9]:
def create_dataset(bs, imgs, chn, trains, vals):
    trilist = [x for r in range(len(w_imgl)) if r in trains for x in w_imgl[r]]
    trmlist = [x for r in range(len(w_mskl)) if r in trains for x in w_mskl[r]]
    valilist = [x for r in range(len(w_imgl)) if r in vals for x in w_imgl[r]]
    valmlist = [x for r in range(len(w_mskl)) if r in vals for x in w_mskl[r]]

    a = list(zip(trilist, trmlist))
    #     rd.shuffle(a)
    trilist, trmlist = zip(*a)

    tr_ds = tf.data.Dataset.from_generator(imggen, (float32, float32),
                                           args=[trilist, trmlist, len(trmlist), imgs, True])
    tr = tr_ds.map(fixx, num_parallel_calls=tf.data.experimental.AUTOTUNE).repeat().batch(bs)

    val_ds = tf.data.Dataset.from_generator(imggen, (float32, float32),
                                            args=[valilist, valmlist, len(valmlist), imgs, False])
    val = val_ds.map(fixx).batch(bs)

    return tr, val

In [10]:
def create_testds(bs, imgs, chn, valn, indx):
    Test_ds = tf.data.Dataset.from_generator(imggen, (float32, float32),
                                             args=[w_imgl[valn[indx]], w_mskl[valn[indx]], len(w_imgl[valn[indx]]),
                                                   imgs, False])
    Testing = Test_ds.map(fixx).batch(bs)
    return Testing


def create_valds(bs, imgs, chn, valn, indx):
    Test_ds = tf.data.Dataset.from_generator(imggen, (float32, float32),
                                             args=[w_imgl[valn[indx]], w_mskl[valn[indx]], len(w_imgl[valn[indx]]),
                                                   imgs, False])

    Testing = Test_ds.map(fixx).batch(bs)
    return Testing


   # Unet Model

In [11]:
def iou_0(y_true, y_pred, smooth=1e-3):
    y_true = K.cast(K.squeeze(K.one_hot(K.cast(y_true, 'int64'), 2), -2), 'float32')
    y_true = tf.slice(y_true, [0, 0, 0, 0], [-1, -1, -1, 1])
    y_pred = tf.slice(y_pred, [0, 0, 0, 0], [-1, -1, -1, 1])
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
    union = K.sum(y_true, [1, 2, 3]) + K.sum(y_pred, [1, 2, 3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou


def iou_1(y_true, y_pred, smooth=1e-3):
    y_true = K.cast(K.squeeze(K.one_hot(K.cast(y_true, 'int64'), 2), -2), 'float32')
    y_true = tf.slice(y_true, [0, 0, 0, 1], [-1, -1, -1, -1])
    y_pred = tf.slice(y_pred, [0, 0, 0, 1], [-1, -1, -1, -1])
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
    union = K.sum(y_true, [1, 2, 3]) + K.sum(y_pred, [1, 2, 3]) - intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou



In [12]:
def kappa(y_true, y_pred, argm=True):
    if argm:
        y_pred2 = tf.reshape(tf.argmax(y_pred, axis=-1), [-1])
    else:
        y_pred2 = tf.reshape(y_pred, [-1])
    y_true2 = tf.reshape(y_true, [-1])
    conf = tf.cast(tf.math.confusion_matrix(y_true2, y_pred2, num_classes=2), float32)
    actual_ratings_hist = tf.reduce_sum(conf, axis=1)
    pred_ratings_hist = tf.reduce_sum(conf, axis=0)

    #print(conf,actual_ratings_hist,pred_ratings_hist)
    nb_ratings = tf.shape(conf)[0]
    weight_mtx = tf.zeros([nb_ratings, nb_ratings], dtype=tf.float32)
    diagonal = tf.ones([nb_ratings], dtype=tf.float32)
    weight_mtx = tf.linalg.set_diag(weight_mtx, diagonal=diagonal)
    gc = actual_ratings_hist * pred_ratings_hist
    conf = tf.cast(conf, float32)
    totaln = tf.cast(tf.shape(y_true2)[0], float32)
    up = tf.cast(totaln * tf.reduce_sum(conf * weight_mtx), float32) - tf.cast(tf.reduce_sum(gc), float32)
    down = tf.cast(totaln ** 2 - tf.reduce_sum(gc), float32)
    #print(weight_mtx,gc,conf,up,down)

    if tf.math.is_nan(up / down):
        return 0.
    return up / down



In [13]:
def custom_cost_function(y, y_pred):
    WD = 10.
    y = K.cast(K.squeeze(K.one_hot(K.cast(y, 'int64'), 2), -2), 'float32')

    return K.sum(y * (-1) * tf.convert_to_tensor([1., WD]) * tf.math.log(y_pred + 1e-9), -1)



In [14]:
def upsample(filters, size):
    return [layers.Conv2DTranspose(filters, size, strides=2, padding='same'), layers.BatchNormalization(),
            layers.ReLU()]



In [15]:
def def_basem():
    base_model = tf.keras.applications.VGG16(input_shape=[imgs, imgs, 3], include_top=False)

    vgg16_layer_names = [
        'block1_conv2',  # 1
        'block2_conv2',  # 1/2
        'block3_conv3',  # 1/4
        'block4_conv3',  # 1/8
        'block5_conv3',  # 1/16
    ]
    vgg16_layer_names.reverse()
    vgglayers = base_model.outputs + [base_model.get_layer(name).output for name in vgg16_layer_names]

    down_stack = tf.keras.Model(inputs=base_model.input, outputs=vgglayers)
    # down_stack.summary()
    return down_stack


def def_unet(classes, height, width, channels, base_model, name):
    inputs = layers.Input(shape=(height, width, channels))
    #xx = layers.Dense(3,activation='sigmoid')(inputs)

    # >3 channels
    #     input1 = layers.Conv2D(channels,(3,3),padding='same')(inputs)
    #     input2 = layers.Conv2D(channels,(3,3),padding='same')(input1)
    #     input = input = layers.Conv2D(3,(1,1),padding='same')(input2)
    #     skips = base_model(input)

    # 3 channels
    skips = base_model(inputs)

    x = skips[0]

    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    for layer in upsample(512, (3, 3)):
        x = layer(x)

    for ch, skip in zip([256, 128, 64, 32], skips[1:-1]):
        x = layers.Concatenate()([x, skip])
        for layer in upsample(ch, (3, 3)):
            x = layer(x)

    x = layers.Concatenate()([x, skips[-1]])
    x = layers.Conv2D(96, (1, 1), padding='same')(x)

    x = layers.Conv2D(classes, (3, 3), padding='same', activation='softmax')(x)
    #x = layers.Dense(classes, activation='sigmoid')(x)

    unet = tf.keras.Model(inputs=inputs, outputs=x, name=name)
    #unet.summary()
    base_model.trainable = True

    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=1e-4,
        decay_steps=10000,
        decay_rate=0.9)

    opt = keras.optimizers.Adam(learning_rate=0.0001)
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    custom_cost_function

    unet.compile(loss=custom_cost_function, optimizer=opt, metrics=['acc', kappa])
    # unet.compile(loss=custom_cost_function, optimizer=opt, metrics=['acc', kappa, iou_0, iou_1])
    print(unet.summary())
    return unet



In [ ]:
# import graphviz, pydot

# down_stack = def_basem()
# model = def_unet(2,imgs,imgs,CH_N,down_stack,'unet16')
# tf.keras.utils.plot_model(model, show_shapes=True)


   # Combine

In [20]:
# COMBINE THE ANNOTATION IMAGES
def combinemsk(whole_mskl):
    co = 0
    for i in range(45):
        for r in range(48):
            msk = tf.io.read_file(whole_mskl[co])
            msk = tf.image.decode_jpeg(msk, channels=1)
            msk = tf.image.convert_image_dtype(msk, tf.float32)
            msk = tf.math.logical_and(msk < 256, msk > 0)
            mskn = msk.numpy().squeeze()
            if r == 0:
                temp = mskn
            else:
                temp = np.concatenate((temp, mskn), axis=0)
            co += 1
        if i == 0:
            compmsk = temp
        else:
            compmsk = np.concatenate((compmsk, temp), axis=1)

    return compmsk



In [21]:
# COMBINE THE AERIAL IMAGES
def combinetif(whole_imgl):
    dst = Image.new('RGBX', (256 * 45, 256 * 48))
    co = 0
    for i in range(45):
        temp = Image.new('RGBX', (256, 256 * 48))
        for r in range(48):
            img2 = Image.open(whole_imgl[co])
            co += 1
            temp.paste(img2, (0, 256 * r))
        dst.paste(temp, (i * 256, 0))
    return dst



In [22]:
# COMBINE THE SEGMENTATION RESULT IMAGES
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import roc_curve, auc, precision_recall_curve


def wholepic(w_ds, model, vstep, threshold=None):
    allm = []
    y_true = []
    for _, m in w_ds.take(vstep):
        # try:
        #   y_true = np.concatenate((y_true,m.numpy()))
        # except:
        #   y_true = m.numpy()
        for r in m:
            allm.append(r)
    wout = model.predict(w_ds, verbose=0, steps=vstep)
    if threshold == None:
        w_output = tf.argmax(wout, axis=3)
    else:
        w_output = wout[:, :, :, 1] > threshold

    # CALCULATE THE SEGMENTATION RESULT MATRICS AFTER COMBINE
    w_output = tf.cast(w_output, tf.int8)
    # ytflat = y_true.flatten().astype('int8')
    # woflat = w_output.numpy().flatten()
    # print(pd.crosstab(ytflat, woflat, rownames = ['label'], colnames = ['predict'])) #CONFUSION MATRIX
    w_output = tf.expand_dims(w_output, -1)
    # los = keras.losses.sparse_categorical_crossentropy(allm,wout)
    t1 = time.time()
    kap = kappa(allm, w_output, False).numpy()
    t2 = time.time()
    print('kap:', kap, 'time:', (t2 - t1))

    del wout, allm

    co = 0
    for i in range(45):
        for r in range(48):
            mskn = w_output[co]
            mskn = tf.squeeze(mskn, -1)
            co += 1
            if r == 0:
                temp = mskn
            else:
                temp = np.concatenate((temp, mskn), axis=0)
        if i == 0:
            compmsk2 = temp
        else:
            compmsk2 = np.concatenate((compmsk2, temp), axis=1)

    # return compmsk2,[kap, accuracy_score(ytflat,woflat), iou0, iou1, pre,rec]
    return compmsk2, [kap]


   # Training

In [23]:
import gc

gc.enable()
from sklearn.utils.extmath import stable_cumsum



In [24]:
def PRC_kappa(tds, vstep, model):
    y_trueo = []
    for _, m in tds.take(vstep):
        try:
            y_trueo = np.concatenate((y_trueo, m.numpy()))
        except:
            y_trueo = m.numpy()
    wout = model.predict(tds, verbose=0, steps=vstep)
    y_true = y_trueo.ravel()
    y_score = wout[:, :, :, 1].ravel()
    del wout
    del y_trueo

    gc.collect()

    ### CALCULATE CONFUSION MATRICS / KAPPA FOR DIFFERENT THRESHOLDS/ MODIFIED FROM SKLEARN 
    y_true = (y_true == 1)
    desc_score_indices = np.argsort(y_score)[::-1]
    #y_score = y_score[desc_score_indices]
    #y_true = y_true[desc_score_indices]
    np.take(y_score, desc_score_indices, out=y_score)
    np.take(y_true, desc_score_indices, out=y_true)
    distinct_value_indices = np.where(np.diff(y_score))[0]
    threshold_idxs = np.r_[distinct_value_indices, y_true.size - 1]
    tps = stable_cumsum(y_true)[threshold_idxs]
    fps = 1 + threshold_idxs - tps
    del distinct_value_indices
    del desc_score_indices

    gc.collect()
    y_true2 = y_true[::-1]
    del y_true
    tns = stable_cumsum(np.invert(y_true2))[::-1][threshold_idxs]
    #fns = stable_cumsum(1-np.invert(y_true2))[::-1][threshold_idxs]
    total = tps[-1] + tps[0] + fps[0] + tns[0]
    fns = total - fps - tns - tps
    del y_true2
    del total

    gc.collect()
    total = (tns + tps + fns + fps)[0]
    po = (tns + tps) / total
    p1 = ((tps + fps) / total) * ((tps + fns) / total)
    p2 = ((tns + fps) / total) * ((tns + fns) / total)
    pe = p1 + p2
    kap = (po - pe) / (1 - pe)

    optthresh = y_score[threshold_idxs][np.argmax(kap)]
    print('Best Kappa: %.3f  when threshold = %.3f ' % (max(kap), optthresh))
    # print('Best Kappa: %.3f  when threshold = %.3f  precision: %.3f  recall: %.3f' % (max(kap),optthresh,precision[np.argmax(kap)],recall[np.argmax(kap)]))

    return optthresh



In [25]:
# OUTPUT EXP RESULT
def writefile(fold, res, rtype):
    res1 = res[0]

    global dst
    workbook = openpyxl.load_workbook(dst)
    global sheet_name
    sheet = workbook[sheet_name]

    if rtype == 'validation':
        shift = 8
    else:
        shift = 4
    for col in range(len(res1)):
        sheet.cell(column=(col + shift), row=(fold + 3), value="%.4f" % res1[col])

    workbook.save(dst)


def writefile_train(fold, trains, vals, hist):
    global dst
    workbook = openpyxl.load_workbook(dst)
    global sheet_name
    sheet = workbook[sheet_name]
    string = 'train:'
    for i in range(len(trains)):
        if i == 0:
            string = string + str(trains[i])
        else:
            string = string + ', ' + str(trains[i])
    string = string + "\nval:{}".format(vals[0])
    sheet.cell(column=2, row=(fold + 3), value=string)

    res = []
    hist = hist.history
    res.append(sum(hist['loss']) / len(hist['loss']))
    res.append(sum(hist['kappa']) / len(hist['kappa']))
    res.append(sum(hist['acc']) / len(hist['acc']))

    shift = 13
    for col in range(len(res)):
        sheet.cell(column=(col + shift), row=(fold + 3), value="%.4f" % res[col])
    workbook.save(dst)



In [27]:

EPOCH_N = 1  #20


# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
def my_train(fold, dirp):
    t1 = time.time()
    for i in fold:
        print("=== FOLD %d ===" % (i + 1))
        bm = def_basem()
        model = def_unet(2, imgs, imgs, CH_N, bm, 'unet')

        NOTE = ('FINAL_5FC4')

        vals = [x for x in range(5) if x % 5 == i]

        #         rest = [x for x in range(4) if x not in tests]
        #         random.shuffle(rest)
        #         vals = rest[:1]
        #         trains = [x for x in rest if x not in vals]

        print('Vals ', vals, '\n Trains ', trains)
        imgtr_ds, imgval_ds = create_dataset(BATCH_SIZE, imgs, CH_N, trains, vals)
        imgtr_ds = imgtr_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

        mc = tf.keras.callbacks.ModelCheckpoint((dirp + r"/Models/U_BS%d_I%d_%s.h5" % (BATCH_SIZE, imgs, NOTE)),
                                                monitor='val_kappa', mode='max', verbose=1, save_best_only=True)
        mc2 = tf.keras.callbacks.ModelCheckpoint((dirp + r"/Models/U_BS%d_I%d_%s" % (BATCH_SIZE, imgs, NOTE)),
                                                 monitor='val_kappa', mode='max', verbose=1, save_best_only=True)
        ulogdir = dirp + r'/logs/U_BS%d_I%d_%s' % (BATCH_SIZE, imgs, NOTE)
        tensorboard_callback = tf.keras.callbacks.TensorBoard(ulogdir, histogram_freq=1)

        VALIDATION_STEPS = math.ceil(len(w_imgl[0]) * 1 / BATCH_SIZE)
        STEPS_PER_EPOCH = math.ceil(len(w_imgl[0]) * 5 / BATCH_SIZE)

        historyt = model.fit(imgtr_ds, epochs=EPOCH_N,
                             steps_per_epoch=STEPS_PER_EPOCH,
                             validation_steps=VALIDATION_STEPS,
                             validation_data=imgval_ds,
                             callbacks=[mc, mc2, tensorboard_callback]  #mc,
                             )
        history.append(historyt)

        model.save(root + 'Models/U_BS%d_I%d_%s' % (BATCH_SIZE, imgs, NOTE))
        tf.keras.backend.clear_session()
        model.load_weights(dirp + r'/Models/U_BS%d_I%d_%s.h5' % (BATCH_SIZE, imgs, NOTE))

        writefile_train(fold[0], trains, vals, historyt)
        #     for t in history:
        #       print(t.history)

        t2 = time.time()

        # VALIDATION
        gc.collect()
        vresult = []
        opvalTH = PRC_kappa(imgval_ds, VALIDATION_STEPS, model)
        custom_TH.append(opvalTH)

        for j in range(1):
            print("Validation %d" % (j + 1))
            plt.figure(figsize=(8, 8))
            plt.rc('font', size=14)
            plt.imshow(combinetif(w_imgl[vals[j]]))# only to show
            plt.show()
            plt.close()

            # OUTPUT COLOR
            aa = np.full([12288, 11520, 3], 255, 'uint8')
            c1 = np.full([12288, 11520, 1], 213, 'uint8')
            c2 = np.full([12288, 11520, 1], 180, 'uint8')
            c3 = np.full([12288, 11520, 1], 60, 'uint8')
            cc = np.concatenate((c1, c2, c3), axis=2)

            # PREDICTION
            tds = create_valds(BATCH_SIZE, imgs, CH_N, vals, j)
            mask = combinemsk(w_mskl[vals[j]]) # only to show
            # I add the batch size

            valresult = model.evaluate(tds, steps=math.ceil(len(w_imgl[0]) / BATCH_SIZE))
            out, met = wholepic(tds, model, math.ceil(len(w_imgl[0]) / BATCH_SIZE),
                                opvalTH)  #met = Metrics of the output
            out1 = (tf.image.resize(np.expand_dims(out, -1), (12288, 11520)) > 0.5).numpy().astype(
                'uint8')  #11520x12288

            # SHOW THE VALIDATION GT AND RESULT
            plt.figure(figsize=(16, 8))
            plt.rc('font', size=14)
            a = plt.subplot(1, 2, 1)
            a.set_title('True Mask')
            a = plt.imshow(mask)
            b = plt.subplot(1, 2, 2)
            b.set_title('Predicted Mask')
            b = plt.imshow(np.squeeze(out1))
            plt.show()
            plt.close()

            # SAVE RESULT the image
            rout = np.where(out1, cc, aa)
            im = Image.fromarray(rout)
            im = im.resize((11460, 12260), Image.NEAREST)
            im.save(dirp + r'/Val/TH_Fold%d_VAL%d_IMG%d.png' % ((i + 1), (j + 1), vals[j]))
            # plt.imsave(root + r'/Fold%d_VAL%d_IMG%d_TH.png' % ((i+1), (j+1), vals[j]), rout)
            del c1, c2, c3, out, out1, rout

            # LOSS + KAP + ACC
            vresult.append([valresult[0]] + met)
            # vresult_p.append([valresult[0]] + met_p)
            writefile(fold[0], vresult, 'validation')

            gc.collect()
        val_result.append(vresult)
        #val_result_o.append(vresult_p)
        print('Val Custom TH', val_result)
        t3 = time.time()
        print("time: ", (t2 - t1), ", ", (t3 - t2))
        print('custom_TH:', custom_TH)


In [28]:
history = []
val_result = []
teresult = []
teresult_p = []
custom_TH = []

   # Start Training

In [ ]:
# print(dirp)
# print(dst, ": ", sheet_name)

trains = [0, 1, 2, 3, 4]
my_train([0], dirp)


In [ ]:

# trains = [2,3]
# vals = [0]
# my_train([1], dirp)


In [ ]:

# trains = [0,3]
# vals = [1]
# my_train([2], dirp)


In [ ]:

# trains = [0,1]
# vals = [2]
# my_train([3], dirp)


   # Result

In [ ]:
print('Val Custom TH', val_result, '\n Test Custom TH', teresult)

In [ ]:
# for t in history:
#   print(t.history)

for i in history:
    for r in i.history.values():
        print('%.3f' % r[-1], end=' ')
#   print()

In [ ]:
valme = [val_result]
teme = [teresult]

In [ ]:
print("Validation\n")
for i in range(1):
    print('Fold %d' % (i + 1))
    # print('Loss Acc Kappa IoU0 IoU1 Preci(0) Recall (0) Preci(1) Recall (1)')
    print('Loss Kappa')
    for l in range(1):
        for t in valme:
            print('%.3f %.3f' % (t[i][l][0], t[i][l][1]))

In [ ]:
# crilist3=['Loss','Accuracy','Kappa','IoU_0','IoU_1']
crilist2 = ['loss', 'acc', 'kappa']
crilist3 = ['Loss', 'Accuracy', 'Kappa']
plt.rc('font', size=20)
for n, r in enumerate(history):
    plt.figure(figsize=(11, 10))
    plt.suptitle('Fold %d Training' % ((n + 1)), fontsize=25)
    for i in crilist2:
        try:
            plt.plot(np.array(r[i]), linewidth=3)
        except:
            plt.plot(np.array(r.history[i]), linewidth=3)
    plt.xticks(range(20), range(1, 21))
    plt.xlabel('Epoch')
    plt.legend(crilist3)
    plt.show()

In [ ]:
# crilist3=['Loss','Accuracy','Kappa','IoU_0','IoU_1']
crilist2 = ['loss', 'acc', 'kappa']
crilist3 = ['Loss', 'Accuracy', 'Kappa']
plt.rc('font', size=20)
for n, r in enumerate(history):
    plt.figure(figsize=(11, 10))
    plt.suptitle('Fold %d Validation' % ((n + 1)), fontsize=25)
    for i in crilist2:
        try:
            plt.plot(np.array(r['val_' + i]), linewidth=3)
        except:
            plt.plot(np.array(r.history['val_' + i]), linewidth=3)
    plt.xticks(range(20), range(1, 21))
    plt.xlabel('Epoch')
    plt.legend(crilist3)
    plt.show()

# Testing

In [ ]:
#Sensitivity Analysis we use 5 images of Real Testing
imgfiles = [f for f in glob.glob(r'/work/u8104687/24_IMG_CROP/*.png')]
mskfiles = [f for f in glob.glob(r'/work/u8104687/24_MASK_CROP/*.png')]

imgfiles.sort()
mskfiles.sort()
print(len(imgfiles), len(mskfiles))

In [ ]:
n_pic = len(imgfiles) // 2160
whole_imgl = [{} for _ in range(n_pic)]
whole_mskl = [{} for _ in range(n_pic)]
for r, i in enumerate(zip(imgfiles, mskfiles)):
    whole_imgl[r % n_pic][int(i[0].split('/')[-1].split('_')[0])] = (i[0])
    whole_mskl[r % n_pic][int(i[0].split('/')[-1].split('_')[0])] = (i[1])
w_imgl = [[x.get(i) for i in range(1, len(x) + 1, 1)] for x in whole_imgl]
w_mskl = [[x.get(i) for i in range(1, len(x) + 1, 1)] for x in whole_mskl]
print(len(w_imgl))

In [ ]:
def my_test(fold, opvalTH, tests, dirp):
    teresult = []
    t1 = time.time()
    for i in fold:
        print("=== FOLD %d ===" % (i + 1))
        bm = def_basem()
        model = def_unet(2, imgs, imgs, CH_N, bm, 'unet')
        NOTE = ('FINAL_5FC4')

        #         model_path = glob.glob(dirp + r'/Models/U_BS64_I256_model{}.h5'.format(int(fold[0])+1))
        model_path = glob.glob(dirp + r'/Models/U_BS64_I256_FINAL_5FC4.h5')

        print("Model: ", model_path[0])
        model.load_weights(model_path[0])

        aa = np.full([12288, 11520, 3], 255, 'uint8')
        c1 = np.full([12288, 11520, 1], 213, 'uint8')
        c2 = np.full([12288, 11520, 1], 180, 'uint8')
        c3 = np.full([12288, 11520, 1], 60, 'uint8')
        cc = np.concatenate((c1, c2, c3), axis=2)
        t3 = time.time()

        # TESTING
        tresult = []
        pmet = []
        omet = []
        for j in range(len(tests)):
            print(j)
            print("Test %d" % (j + 1))
            plt.figure(figsize=(8, 8))
            plt.rc('font', size=14)
            plt.imshow(combinetif(w_imgl[tests[j]]))
            plt.show()
            plt.close()

            tds = create_testds(BATCH_SIZE, imgs, CH_N, tests, j)
            #       result = model.evaluate(tds, steps = math.ceil(len(w_imgl[0])/BATCH_SIZE))
            out, met = wholepic(tds, model, math.ceil(len(w_imgl[0]) / BATCH_SIZE), opvalTH)

            # SHOW THE TESTING GT AND RESULT
            plt.figure(figsize=(18, 18))
            plt.rc('font', size=14)
            a = plt.subplot(1, 2, 1)
            a.set_title('True Mask')
            allmask = combinemsk(w_mskl[tests[j]])
            a = plt.imshow(allmask)
            b = plt.subplot(1, 2, 2)
            b.set_title('Predicted Original')

            out1 = (tf.image.resize(np.expand_dims(out, -1), (12288, 11520)) > 0.5).numpy().astype(
                'uint8')  #11460 x #12260
            b = plt.imshow(np.squeeze(out))
            plt.show()
            plt.close()

            rout = np.where(out1, cc, aa)
            plt.imsave(dirp + '/Test/TH_Fold%d_REAL TEST JIAYI%d.png' % ((i + 1), (j + 1)), rout)
            del out1, rout

            # ------------------------------------------------------------------------------------------------------------------------
            # KERNEL 10
            close10 = cv2.morphologyEx(np.uint8(out), cv2.MORPH_OPEN,
                                       cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))  #Remove
            out_p10 = cv2.morphologyEx(np.uint8(close10), cv2.MORPH_CLOSE,
                                       cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10)))  #Close
            del out, close10

            plt.figure(figsize=(18, 18))
            plt.rc('font', size=14)
            a = plt.subplot(1, 2, 1)
            a.set_title('True Mask')
            # allmask = combinemsk(w_mskl[tests[j]])
            a = plt.imshow(allmask)
            b = plt.subplot(1, 2, 2)
            b.set_title('Predicted Post Processing 10')
            b = plt.imshow(np.squeeze(out_p10))
            plt.show()
            plt.close()

            pmet_10 = kappa(allmask, out_p10, False).numpy()
            # omet = kappa(allmask, out, False).numpy()
            print('KAPPA Post Processing 10 = %.3f' % pmet_10)
            print('KAPPA Original = %.3f' % met[0])
            #       met.insert(1, pmet_10)

            out10 = (tf.image.resize(np.expand_dims(out_p10, -1), (12288, 11520)) > 0.5).numpy().astype(
                'uint8')  #11460 x #12260
            rout10 = np.where(out10, cc, aa)
            plt.imsave(dirp + '/Test/TH_Fold%d_REAL TEST JIAYI%d_IMG%d_Post10.png' % ((i + 1), (j + 1), tests[j]),
                       rout10)
            plt.clf()

            tresult.append(pmet_10)

        #         tresult.append((tresult[0]+tresult[1])/2)
        tresult.append((tresult[0] + tresult[1] + tresult[2] + tresult[3]) / 4)
        #         writefile(fold[0], [tresult], 'testing')

        gc.collect()
        teresult.append(tresult)
        print('Test Custom TH', teresult)
        t4 = time.time()
        print("time: ", (t4 - t3))




In [ ]:
#Real Testing for MODEL 1
optthresh = custom_TH[0]
my_test([0], optthresh, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], dirp)

In [ ]:
# #Real Testing for FOLD 2
# optthresh=custom_TH[1]
# my_test([1], optthresh, [0,1,2,3,4], dirp)

In [ ]:
# #Real Testing for FOLD 3
# optthresh=custom_TH[2]
# my_test([2], optthresh, [0,1,2,3,4], dirp)

In [ ]:
# #Real Testing for FOLD 4
# optthresh=custom_TH[3]
# my_test([3], optthresh, [0,1,2,3,4], dirp)

In [ ]:
# #COA REAL TESTING 59

# imgfiles = [f for f in glob.glob(r'/work/u8104687/IMG_COA_REAL_TEST/*.png')]
# mskfiles = [f for f in glob.glob(r'/work/u8104687/MASK_COA_REAL_TEST/*.png')]
# imgfiles.sort()
# mskfiles.sort()
# print(len(imgfiles),len(mskfiles))

In [ ]:
# n_pic = len(imgfiles)//2160
# whole_imgl = [ {} for _ in range(n_pic)]
# whole_mskl = [ {} for _ in range(n_pic)]
# for r,i in enumerate(zip(imgfiles,mskfiles)):
#     whole_imgl[r%n_pic][int(i[0].split('/')[-1].split('_')[0])]=(i[0])
#     whole_mskl[r%n_pic][int(i[0].split('/')[-1].split('_')[0])]=(i[1])
# w_imgl = [[x.get(i) for i in range(1,len(x)+1,1)] for x in whole_imgl]
# w_mskl = [[x.get(i) for i in range(1,len(x)+1,1)] for x in whole_mskl]
# print(len(w_imgl))

In [ ]:
# optthresh=custom_TH[3]
# my_test([3], optthresh, [0], dirp)